In [46]:
import pandas as pd
import numpy as np

In [47]:
description = pd.read_csv('dane_archiwalne_meteo/kody_parametr.csv', encoding='cp1250', delimiter=';')
geo_loc = pd.read_csv('dane_archiwalne_meteo/kody_stacji.csv', encoding='cp1250', delimiter=';')

In [48]:
description.drop([5, 6, 7], inplace=True)  # rain is taken in another source
description

,Kod,Nazwa
0,B00300S,Temperatura powietrza (oficjalna)
1,B00305A,Temperatura gruntu (czujnik)
2,B00202A,Kierunek wiatru (czujnik)
3,B00702A,Średnia prędkość wiatru czujnik 10 minut
4,B00703A,Prędkość maksymalna (czujnik)
8,B00802A,Wilgotność względna powietrza (czujnik)
9,B00714A,Największy poryw w okresie 10min ze stacji Syn...
10,B00910A,Zapas wody w śniegu (obserwator)


In [49]:
geo_loc = geo_loc[['ID', 'Nazwa', 'Szerokość geograficzna', 'Długość geograficzna', 'Wysokość n.p.m.']]
geo_loc.head()

,ID,Nazwa,Szerokość geograficzna,Długość geograficzna,Wysokość n.p.m.
0,250180590,RYBNIK-STODOŁY,50 09 16,18 28 59,216
1,254140010,DZIWNÓW,54 01 20,14 43 54,7
2,250160530,ZIELENIEC,50 19 48,16 23 39,840
3,250160630,MIĘDZYGÓRZE,50 13 06,16 46 23,800
4,250160650,MIĘDZYLESIE,50 09 12,16 40 15,453


In [50]:
dti = pd.date_range('2022-01-01', '2023-01-01', freq='10min')
dti

DatetimeIndex(['2022-01-01 00:00:00', '2022-01-01 00:10:00',
               '2022-01-01 00:20:00', '2022-01-01 00:30:00',
               '2022-01-01 00:40:00', '2022-01-01 00:50:00',
               '2022-01-01 01:00:00', '2022-01-01 01:10:00',
               '2022-01-01 01:20:00', '2022-01-01 01:30:00',
               ...
               '2022-12-31 22:30:00', '2022-12-31 22:40:00',
               '2022-12-31 22:50:00', '2022-12-31 23:00:00',
               '2022-12-31 23:10:00', '2022-12-31 23:20:00',
               '2022-12-31 23:30:00', '2022-12-31 23:40:00',
               '2022-12-31 23:50:00', '2023-01-01 00:00:00'],
              dtype='datetime64[ns]', length=52561, freq='10T')

In [51]:
parameters = description[['Kod']].to_numpy().reshape(-1)  # columns
dti = pd.date_range('2022-01-01', '2023-01-01', freq='10min')  # indexes as timestamp

data_for_cracow = pd.DataFrame(columns=parameters, index=dti)

# data_for_cracow.xs('2022-01-01 00:50:00')['B00300S'] = 1
data_for_cracow

,B00300S,B00305A,B00202A,B00702A,B00703A,B00802A,B00714A,B00910A
2022-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-01 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-12-31 23:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
station_codes_near_cracow = [350190566, 250190470]

year = 2022
for month in range(1, 13):
    for param_code in parameters:
        path = f'dane_archiwalne_meteo/data/{param_code}/{param_code}_{year}_{month:02d}.csv'
        single_parameter_per_month_data = pd.read_csv(path, delimiter=';', header=None)
        
        single_parameter_per_month_data = single_parameter_per_month_data[[0, 1, 2, 3]]
        
        # take data from one station near Cracow
        single_parameter_per_cracow = single_parameter_per_month_data.where(single_parameter_per_month_data[0] == station_codes_near_cracow[0])
        
        single_parameter_per_cracow.dropna(inplace=True)
        
        if len(single_parameter_per_cracow.index) == 0:
            single_parameter_per_cracow = single_parameter_per_month_data.where(single_parameter_per_month_data[0] == station_codes_near_cracow[1])
        
        single_parameter_per_cracow.dropna(inplace=True)
        
        dates = single_parameter_per_cracow[2]
        dates = pd.to_datetime(dates)
        values = single_parameter_per_cracow[3]
        
        for i in dates.index:
            data_for_cracow.xs(dates[i])[param_code] = values[i]


C:\Users\Tomek\AppData\Local\Temp\ipykernel_14172\4275870745.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  single_parameter_per_month_data = pd.read_csv(path, delimiter=';', header=None)
C:\Users\Tomek\AppData\Local\Temp\ipykernel_14172\4275870745.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  single_parameter_per_month_data = pd.read_csv(path, delimiter=';', header=None)
C:\Users\Tomek\AppData\Local\Temp\ipykernel_14172\4275870745.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  single_parameter_per_month_data = pd.read_csv(path, delimiter=';', header=None)
C:\Users\Tomek\AppData\Local\Temp\ipykernel_14172\4275870745.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  single_parameter_per_month_data = pd.read_csv(path, delimiter=';', header=None)
C:\U

In [53]:
i = int('4.3')

ValueError: invalid literal for int() with base 10: '4.3'

In [54]:
data_for_cracow

,B00300S,B00305A,B00202A,B00702A,B00703A,B00802A,B00714A,B00910A
2022-01-01 00:00:00,"9,5","9,21",234,"6,6","5,5",93,NaN,NaN
2022-01-01 00:10:00,"9,4","9,05",232,"6,2","6,8",93,NaN,NaN
2022-01-01 00:20:00,"9,2","8,93",227,"5,7","5,6",94,NaN,NaN
2022-01-01 00:30:00,"9,2","8,97",223,"5,5","7,3",95,NaN,NaN
2022-01-01 00:40:00,"9,1","9,05",225,"5,1","6,3",95,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-12-31 23:20:00,"5,5","4,74",184,"0,6","1,2",87,NaN,NaN
2022-12-31 23:30:00,"5,6","4,48",199,"0,7","1,4",88,NaN,NaN
2022-12-31 23:40:00,"5,8","4,38",0,0,"1,2",89,NaN,NaN
2022-12-31 23:50:00,"5,4","4,31",0,0,"1,1",89,NaN,NaN


In [58]:
data_for_cracow.B00714A.count()

143

In [59]:
data_for_cracow.B00910A.count()

26

In [61]:
data_for_cracow.to_csv('arch_meteo_krakow.csv')